In [2]:
from z3 import *

We are going to verify this program:
```
assume(B > 0);
C = A;
D = 0;
while (D < B) {
    C++; D++;
}
assert(C == A + B);
```

In [3]:
_int = IntSort(); _bool = BoolSort()
Inv = Function("Inv", _int, _int, _int, _int, _bool)
A, B, C, D, C1, D1 = Ints("A B C D C1 D1")

PRECONDITION = And(B > 0, C == A, D == 0)
POSTCONDITION = (C == A + B)

In [6]:
fp = Fixedpoint()

fp.declare_var(A, B, C, D, C1, D1)
fp.register_relation(Inv)

# B > 0 ∧ C = A ∧ D = 0 → Inv
fp.rule(Inv(A, B, C, D), PRECONDITION)

# Inv ∧ D < B ∧ C' = C + 1 ∧ D' = D + 1 → Inv
fp.rule(Inv(A, B, C1, D1), [Inv(A, B, C, D), D < B, C1 == C + 1, D1 == D + 1])

# Inv ∧ ¬(D < B) → C = A + B
fp.query(And(Inv(A, B, C, D), Not(D < B), Not(POSTCONDITION)))
fp.get_answer()

ForAll([A, B, C, D],
       Inv(A, B, C, D) ==
       And(C + -1*A + -1*D <= 0,
           C + -1*A + -1*B <= 0,
           A + D + -1*C <= 0))

#### Example with an error
We change the postcondition to an incorrect one, $C = A + B + 1$.

(We also force $B > 4$ just so that the result is more interesting for this demonstration)

In [7]:
PRECONDITION = And(B > 4, C == A, D == 0)
POSTCONDITION = (C == A + B + 1)

In [9]:
fp = Fixedpoint()

fp.declare_var(A, B, C, D, C1, D1)
fp.register_relation(Inv)

# B > 0 ∧ C = A ∧ D = 0 → Inv
fp.rule(Inv(A, B, C, D), PRECONDITION)

# Inv ∧ D < B ∧ C' = C + 1 ∧ D' = D + 1 → Inv
fp.rule(Inv(A, B, C1, D1), [Inv(A, B, C, D), D < B, C1 == C + 1, D1 == D + 1])

# Inv ∧ ¬(D < B) → C = A + B
fp.query(And(Inv(A, B, C, D), Not(D < B), Not(POSTCONDITION)))
fp.get_answer()

mp(hyper-res(asserted(ForAll([A, B, C, D],
                             Implies(And(Inv(A, B, C, D),
                                        B <= D,
                                        Not(C == 1 + A + B)),
                                     query!61(A, B, C, D)))),
             hyper-res(asserted(ForAll([A, B, C, D, E, F],
                                       Implies(And(Inv(A,
                                        B,
                                        C,
                                        D),
                                        E == 1 + C,
                                        Not(B <= D),
                                        F == 1 + D),
                                        Inv(A, B, E, F)))),
                       hyper-res(asserted(ForAll([A,
                                        B,
                                        C,
                                        D,
                                        E,
                                        F],
                                        Implies(And(Inv(A,
                                        B,
                                        C,
                                        D),
                                        E == 1 + C,
                                        Not(B <= D),
                                        F == 1 + D),
                                        Inv(A, B, E, F)))),
                                 hyper-res(asserted(ForAll([A,
                                        B,
                                        C,
                                        D,
                                        E,
                                        F],
                                        Implies(And(Inv(A,
                                        B,
                                        C,
                                        D),
                                        E == 1 + C,
                                        Not(B <= D),
                                        F == 1 + D),
                                        Inv(A, B, E, F)))),
                                        hyper-res(asserted(ForAll([A,
                                        B,
                                        C,
                                        D,
                                        E,
                                        F],
                                        Implies(And(Inv(A,
                                        B,
                                        C,
                                        D),
                                        E == 1 + C,
                                        Not(B <= D),
                                        F == 1 + D),
                                        Inv(A, B, E, F)))),
                                        hyper-res(asserted(ForAll([A,
                                        B,
                                        C,
                                        D,
                                        E,
                                        F],
                                        Implies(And(Inv(A,
                                        B,
                                        C,
                                        D),
                                        E == 1 + C,
                                        Not(B <= D),
                                        F == 1 + D),
                                        Inv(A, B, E, F)))),
                                        hyper-res(asserted(ForAll([A,
                                        B],
                                        Implies(Not(B <= 4),
                                        Inv(A, B, A, 0)))),
                                        Inv(-5, 5, -5, 0)),
                                        Inv(-5, 5, -4, 1)),
                                        Inv(-5, 5, -3, 2)),
                                        Inv(-5, 5, -2, 3)),
                                 Inv(-5

**Whoa, what's going on here.** What Z3 (Spacer) produced is a _proof of reachability_ of the query.

Let's try to print it (btw this is the same kind of answer you get if you run e.g. `z3 fp.print_answer=true horn2-error.smt2`).

In [11]:
print(fp.get_answer())

mp(hyper-res(asserted(ForAll([A, B, C, D],
                             Implies(And(Inv(A, B, C, D),
                                        B <= D,
                                        Not(C == 1 + A + B)),
                                     query!61(A, B, C, D)))),
             hyper-res(asserted(ForAll([A, B, C, D, E, F],
                                       Implies(And(Inv(A,
                                        B,
                                        C,
                                        D),
                                        E == 1 + C,
                                        Not(B <= D),
                                        F == 1 + D),
                                        Inv(A, B, E, F)))),
                       hyper-res(asserted(ForAll([A,
                                        B,
                                        C,
                                        D,
                                        E,
                                     

Can you see the trace leading to the error -- i.e. violation of the postcondition in our case?
 * Inv(-5, 5, -5, 0)
 * Inv(-5, 5, -4, 1)
 * Inv(-5, 5, -3, 2)
 * Inv(-5, 5, -2, 3)
 * Inv(-5, 5, -1, 4)
 * Inv(-5, 5,  0, 5)
 * query(-5, 5, 0, 5)
 * False

This is in fact a proper Z3 expression, you can traverse it with the API if you like.

In [12]:
answer = fp.get_answer()
type(answer)

z3.z3.ExprRef

In [38]:
def walk_expr(e):
    yield e
    for c in e.children():
        for x in walk_expr(c): yield x
            
def fv(e):
    """Returns the list of variables in expression `e`"""
    return set(x for x in walk_expr(e) if is_var(x))
            
[x for x in walk_expr(answer) if is_app(x) and x.decl() == Inv and fv(x) == set()]

[Inv(-5, 5, -5, 0),
 Inv(-5, 5, -4, 1),
 Inv(-5, 5, -3, 2),
 Inv(-5, 5, -2, 3),
 Inv(-5, 5, -1, 4),
 Inv(-5, 5, 0, 5)]